In [1]:
import numpy as np
import pickle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams.update({
    'font.size': 14,  # Base font size
    'axes.titlesize': 16,  # Title font size
    'axes.labelsize': 14,  # Axis label size
    'xtick.labelsize': 12,  # X-axis tick labels
    'ytick.labelsize': 12,  # Y-axis tick labels
    'legend.fontsize': 12,  # Legend font size
    'figure.titlesize': 18  # Figure title size
})


In [2]:
# Function to format numbers to 3 significant figures
def format_3_sig_figs(x):
    return '{:.3g}'.format(float(x))

# Function to process data and create dataframe
def process_data(data, grid):
    df = pd.DataFrame(data)
    df['spir'] = grid #np.mean(grid[:,-20:], axis=1)
    return df

# Function to create heatmap
def create_heatmap(data, ax, title, x_axis='',y_axis='sigma', annotate=True):
    grouped_df = data.groupby([y_axis, 'w_scale'])['spir'].mean().reset_index()
    grouped_df = grouped_df.sort_values([y_axis, 'w_scale'])
    grouped_df['noise_formatted'] = grouped_df[y_axis].apply(format_3_sig_figs)
    grouped_df['w_scale_formatted'] = grouped_df['w_scale'].apply(format_3_sig_figs)
    
    pivot_table = grouped_df.pivot(index=y_axis, columns='w_scale', values='spir')
    pivot_table = pivot_table.sort_index(ascending=False)
    pivot_table = pivot_table.sort_index(axis=1)
    
    pivot_table_formatted = pivot_table.applymap(format_3_sig_figs)
    mask = np.isnan(pivot_table.values)
    if annotate:
        sns.heatmap(pivot_table, annot=pivot_table_formatted, fmt='', cmap='YlOrRd', 
                mask=mask, vmax=.8, ax=ax)
    else:
        sns.heatmap(pivot_table, cmap='YlOrRd', 
                mask=mask, vmax=.8, ax=ax)
    
    ax.set_xticklabels([format_3_sig_figs(label.get_text()) for label in ax.get_xticklabels()],
                       rotation=45, ha='right')
    ax.set_yticklabels([format_3_sig_figs(value) for value in pivot_table.index],
                       rotation=0)
    ax.set_title(title)


def create_heatmap(data, ax, title, x_axis='', y_axis='sigma', annotate=True, 
                   normalize_by_seed=False, w_scale_col='w_scale', 
                   use_formatted_columns=False):
    """
    Create a heatmap with optional data preprocessing steps.
    
    Args:
        data: DataFrame containing the data
        ax: matplotlib axis to plot on
        title: title for the plot
        x_axis: x-axis label (default: '')
        y_axis: y-axis label (default: 'sigma')
        annotate: whether to show value annotations (default: True)
        normalize_by_seed: whether to normalize w_scale within seed groups (default: False)
        w_scale_col: name of the w_scale column to use (default: 'w_scale')
        use_formatted_columns: whether to use additional formatting steps (default: False)
    """
    # Make a copy to avoid modifying original data
    cdf = data.copy()
    
    # Normalize w_scale within seed groups if requested
    if normalize_by_seed:
        cdf[w_scale_col] = cdf.groupby('seed')[w_scale_col].transform(
            lambda x: (x - x.min()) / (x.max() - x.min()))
    
  
    # Simplified grouping logic
    grouped_df = cdf.groupby([y_axis, w_scale_col])['spir'].mean().reset_index()
    grouped_df = grouped_df.sort_values([y_axis, w_scale_col])
    
    # Standard pivot
    pivot_table = grouped_df.pivot(index=y_axis, 
                                 columns=w_scale_col, 
                                 values='spir')

    # Sort indices
    pivot_table = pivot_table.sort_index(ascending=False)
    pivot_table = pivot_table.sort_index(axis=1)
    
    # Format values for annotation if needed
    pivot_table_formatted = pivot_table.applymap(format_3_sig_figs)
    mask = np.isnan(pivot_table.values)
    
    # Create heatmap
    if annotate:
        sns.heatmap(pivot_table, annot=pivot_table_formatted, fmt='', 
                    cmap='YlOrRd', mask=mask, vmax=.8, ax=ax)
    else:
        sns.heatmap(pivot_table, cmap='YlOrRd', mask=mask, vmax=.8, ax=ax)
    
    # Format tick labels
    ax.set_xticklabels([format_3_sig_figs(label.get_text()) for label in ax.get_xticklabels()],
                       rotation=45, ha='right')
    ax.set_yticklabels([format_3_sig_figs(value) for value in pivot_table.index],
                       rotation=0)
    ax.set_title(title)



### Isotropic vs Circular Bias connectivitiy Fig 10 K

In [3]:
grid_circ = np.load('outputs/gridsweep_circ.npy')
with open('outputs/paramsets_circ.pkl', 'rb') as f:
    data_circ = pickle.load(f)
    
# Load iso data
grid_iso = np.load('outputs/gridsweep_iso.npy')
with open('outputs/paramsets_iso.pkl', 'rb') as f:
    data_iso = pickle.load(f)

df_circ = process_data(data_circ, grid_circ)
df_iso = process_data(data_iso, grid_iso)
seeds = df_circ['seed'].unique()
seeds

array([ 22,  31,  89, 100, 148, 264])

In [4]:
# Get unique seeds
seeds = df_circ['seed'].unique()
# Create PDF
pdf = PdfPages('figures/sweep_comparison.pdf')
# Create plots for each seed
for seed in seeds:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
    
    # Circular data
    cdf_circ = df_circ.loc[df_circ['seed'] == seed]
    create_heatmap(cdf_circ, ax1, f'Circular (Seed: {seed})', y_axis='noise')
    # Iso data
    cdf_iso = df_iso.loc[df_iso['seed'] == seed]
    create_heatmap(cdf_iso, ax2, f'Iso (Seed: {seed})',  y_axis='noise')
    plt.tight_layout()
    pdf.savefig(fig)
    plt.close(fig)

pdf.close()

C:\Users\xander\AppData\Local\Temp\ipykernel_13628\363291851.py:78: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pivot_table_formatted = pivot_table.applymap(format_3_sig_figs)
C:\Users\xander\AppData\Local\Temp\ipykernel_13628\363291851.py:78: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pivot_table_formatted = pivot_table.applymap(format_3_sig_figs)
C:\Users\xander\AppData\Local\Temp\ipykernel_13628\363291851.py:78: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pivot_table_formatted = pivot_table.applymap(format_3_sig_figs)
C:\Users\xander\AppData\Local\Temp\ipykernel_13628\363291851.py:78: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pivot_table_formatted = pivot_table.applymap(format_3_sig_figs)
C:\Users\xander\AppData\Local\Temp\ipykernel_13628\363291851.py:78: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame

In [5]:
# Circular data

cdf_iso['w_scale_normalized'] = cdf_iso.groupby('seed')['w_scale'].transform(lambda x: (x - x.min()) / (x.max() - x.min()))

cdf_circ['w_scale_normalized'] = cdf_circ.groupby('seed')['w_scale'].transform(lambda x: (x - x.min()) / (x.max() - x.min()))


fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
create_heatmap(cdf_circ, ax1, title='Circular Data', 
               normalize_by_seed=True, 
               use_formatted_columns=True,
               w_scale_col='w_scale_normalized',
               y_axis='noise',
               annotate=False)

# Iso data
create_heatmap(cdf_iso, ax2, title='Iso Data',
               normalize_by_seed=True,
               use_formatted_columns=True,
               w_scale_col='w_scale_normalized',
               y_axis='noise',
               annotate=False)

fig.savefig('figures/circ_v_iso_fig.svg', bbox_inches='tight')

C:\Users\xander\AppData\Local\Temp\ipykernel_13628\938966959.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdf_iso['w_scale_normalized'] = cdf_iso.groupby('seed')['w_scale'].transform(lambda x: (x - x.min()) / (x.max() - x.min()))
C:\Users\xander\AppData\Local\Temp\ipykernel_13628\938966959.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdf_circ['w_scale_normalized'] = cdf_circ.groupby('seed')['w_scale'].transform(lambda x: (x - x.min()) / (x.max() - x.min()))
C:\Users\xander\AppData\Local\Temp\i

### Sweep Sigma Fig 10I Left Panel

In [6]:
seeds = [22, 89, 75, 211, 225, 100, 251,  252, 313, 364, 377, 390 ]
# Create PDF
pdf = PdfPages('figures/sweep_sigma_comparison.pdf')
all_cdfs = []
# Create plots for each seed
for seed in seeds:
    # Load circular data
    grid_circ = np.load(f'outputs/sig_gridsweep_circ_{seed}.npy')
    with open(f'outputs/sig_paramsets_circ_{seed}.pkl', 'rb') as f:
        data_circ = pickle.load(f)
        
    # Create dataframes
    df_circ = process_data(data_circ, grid_circ)

    fig = plt.figure(figsize=(10,10))
    ax1 = fig.gca()
    # Circular data
    cdf_circ = df_circ.loc[df_circ['seed'] == seed]
    all_cdfs.append(cdf_circ)
    create_heatmap(cdf_circ, ax1, f'Circular (Seed: {seed})')
    
    plt.tight_layout()
    pdf.savefig(fig)
    plt.close(fig)

pdf.close()

C:\Users\xander\AppData\Local\Temp\ipykernel_13628\363291851.py:78: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pivot_table_formatted = pivot_table.applymap(format_3_sig_figs)
C:\Users\xander\AppData\Local\Temp\ipykernel_13628\363291851.py:78: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pivot_table_formatted = pivot_table.applymap(format_3_sig_figs)
C:\Users\xander\AppData\Local\Temp\ipykernel_13628\363291851.py:78: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pivot_table_formatted = pivot_table.applymap(format_3_sig_figs)
C:\Users\xander\AppData\Local\Temp\ipykernel_13628\363291851.py:78: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pivot_table_formatted = pivot_table.applymap(format_3_sig_figs)
C:\Users\xander\AppData\Local\Temp\ipykernel_13628\363291851.py:78: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame

In [8]:
df  = pd.concat(all_cdfs)
weak_spiral_seeds = [390, 377, 364,313, 252,211, 75]
strong_spiral_seeds = [89, 100, 225, 251]


fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
create_heatmap(df.loc[df['seed'].isin(weak_spiral_seeds)], ax1, title='weak spirals', 
               normalize_by_seed=True, 
               use_formatted_columns=True,
               w_scale_col='w_scale',
               annotate=False)

# Iso data
create_heatmap(df.loc[df['seed'].isin(strong_spiral_seeds)], ax2, title='strong spiral',
               normalize_by_seed=True,
               use_formatted_columns=True,
               w_scale_col='w_scale',
               annotate=False)

fig.savefig('figures/sigma_fig.svg', bbox_inches='tight')

C:\Users\xander\AppData\Local\Temp\ipykernel_13628\363291851.py:78: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pivot_table_formatted = pivot_table.applymap(format_3_sig_figs)
C:\Users\xander\AppData\Local\Temp\ipykernel_13628\363291851.py:78: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pivot_table_formatted = pivot_table.applymap(format_3_sig_figs)


### Fig 10I right panel

In [9]:
mask = (np.isclose(df['w_scale'].values, 1.5, rtol=.4)) | (np.isclose(df['w_scale'].values, 2.2, rtol=.4)) #| (np.isclose(df['w_scale'].values, 3.3, rtol=.4))
df = df.loc[mask]

In [10]:
fig = plt.figure()
sns.lineplot(x='sigma',y='spir', data=df.loc[df['seed'].isin(weak_spiral_seeds)], errorbar='ci')
fig.savefig('figures/weak_sigma_spir.svg', bbox_inches='tight')

In [11]:
from scipy import stats
weak_df = df.loc[df['seed'].isin(weak_spiral_seeds)]
stats.pearsonr(weak_df['spir'],weak_df['sigma'])

PearsonRResult(statistic=np.float64(0.47487336740969455), pvalue=np.float64(3.0679973226758654e-09))

In [12]:
fig = plt.figure()
sns.lineplot(x='sigma',y='spir', data=df.loc[df['seed'].isin(weak_spiral_seeds)], errorbar='ci', label='weak spiral seeds')
sns.lineplot(x='sigma',y='spir', data=df.loc[df['seed'].isin(strong_spiral_seeds)], errorbar='ci', label='strong spiral seeds')
plt.legend()
fig.savefig('figures/strong_weak_sigma_spir.svg', bbox_inches='tight')

